<img src="https://valueguard.se/static/media/valueguardlogo_black.f3a4c174.png" alt="logo" width="400"/>

# Residential Registry Markups

* Based on official data from the Swedish Land Registry (_Lantmäteriet_), but managed and improved by Valueguard and available through our easy to use API.
* The residential registry contains more than 5.6 million homes in Sweden, of which **2.1 million** are houses and **2.7 million** are apartments. 
* We have correctly coded **85** percent of all tenancies and **99** percent of all cooperative housing.
* The variable "construction_year" is missing on **9** percent of houses (2-4 percent missing for houses built 1980 and earlier) and **30** percent of apartments, a total of **22** percent of all homes
* We update the registry every quarter
* The registry is complete with number of rooms and kitchen type for apartments
* The matching percentage of sold homes to the registry is **94.3** percent for sales within the last 12 months and **97.5** percent for all older sales

# Install dependencies

In [31]:
!pip3 install git+https://github.com/Valueguard-Index-Sweden/valueguard-python-client#egg=valueguard
!pip3 install geopandas

  Cloning https://github.com/Valueguard-Index-Sweden/valueguard-python-client to /tmp/pip-install-alt2iy18/valueguard_7c843524cd4b4c349e530c14ae648f0e
  Running command git clone -q https://github.com/Valueguard-Index-Sweden/valueguard-python-client /tmp/pip-install-alt2iy18/valueguard_7c843524cd4b4c349e530c14ae648f0e
     |████████████████████████████████| 94 kB 3.6 MB/s eta 0:00:011


## Imports

In [1]:
import os
import valueguard
import pandas as pd
import geopandas as gpd

## Settings

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

# Authenticate 

In [7]:
# Create object
vgClient = valueguard.Client()

valueguard_api_username = os. environ['VALUEGUARD_API_USERNAME']
valueguard_api_password = os. environ['VALUEGUARD_API_PASSWORD']

# Generate a access token
vgClient.authenticate(valueguard_api_username,valueguard_api_password)

## Residential registry parameters

In [8]:
residential_registry_parameters = {
    "id":"",
    "address":"",
    "apartment_area_max":"",
    "apartment_area_min":"",
    "apartment_category":"",
    "apartment_nr":"1202",
    "area_id":"",
    "building_purpose":"",
    "construction_year_max":"",
    "construction_year_min":"",
    "floor_max":"",
    "floor_min":"",
    "floors_max":"",
    "floors_min":"",
    "kitchen_type":"",
    "living_space_max":"",
    "living_space_min":"",
    "owner_id":"",
    "postal_area":"",
    "postal_code":"",
    "rooms_max":"",
    "rooms_min":"",
    "updated_at_min":"",
    "street":"Ingemarsgatan",
    "letter":"",
    "number":"7",
    "property_key":"",
    "ownership_type":"",
    "building_type":"",
    "county_code":"",
    "municipality_code":"",
    "parish_code":""
}

residential_registry_limit = 10
residential_registry_i = 0



# Get data

In [9]:
search_criteria={}

print("Usead search parameters:")
for parameter_name in residential_registry_parameters:
    if residential_registry_parameters[parameter_name] != "":
        print("[" + parameter_name + "] => " + residential_registry_parameters[parameter_name])
        search_criteria[parameter_name] = residential_registry_parameters[parameter_name]

residential_registry_df = pd.DataFrame()

while True:
    data = vgClient.residential_registry(offset=residential_registry_i, limit=residential_registry_limit,search_criteria=search_criteria)
    residential_registry_df = residential_registry_df.append(pd.DataFrame(data['residences']))
    fetched_data = data['meta_data']['offset'] + data['meta_data']['response_rows']
    if data['meta_data']['total_nr_records'] <= data['meta_data']['offset'] + data['meta_data']['response_rows']:
        break
    i += data['meta_data']['response_rows']

Usead search parameters:
[apartment_nr] => 1202
[street] => Ingemarsgatan
[number] => 7


In [10]:
residential_registry_df[:5]

,area_name,rooms,building_purpose,rt90x,county_code,owner_id,ownership_type,municipality_code,property_key,area_id,vacation_like_home,apartment_category,number,floors,updated_at,street,wgs84lat,home_type,id,floor,postal_area,sweref99x,sweref99y,address,updated_from,apartment_nr,parish_code,person_registered,probability_deleted,construction_year,deleted,kitchen_type,wgs84lon,apartment_area,letter,error_code,house_nr,postal_code,rt90y,type_code,building_type
0,Stockholm__Sibirien,1.0,1,6583088.0,01,702001-8276,bostadsrätt,80,010131936,5627,1,1,7,5.0,2020-08-11,Ingemarsgatan,59.349076,lägenhet,5034311,3.0,Stockholm,6582913,673805,"Ingemarsgatan 7, 11354",public_data,1202,04,0,0.0,None,0,1,18.056892,35,,0,1,11354,1628092.0,320,33


## Select resident

In [11]:
residential_registry_id_for_markups = residential_registry_df.iloc[0]['id']

## Residential registry markups parameters

In [12]:
residential_registry_markups_limit = 20000
residential_registry_markups_i = 0

## Get markups data

In [13]:
residential_registry_markups_df = pd.DataFrame()

while True:
    data = vgClient.residential_registry_markups(offset=residential_registry_markups_i, limit=residential_registry_markups_limit,
                                                 search_criteria={"id": residential_registry_id_for_markups})
    residential_registry_markups_df.append(pd.DataFrame(data['residences']))
    if data['meta_data']['total_nr_records'] <= data['meta_data']['offset'] + data['meta_data']['response_rows']:
        break
    i += data['meta_data']['response_rows']

In [14]:
residential_registry_markups_df

""
